**Using BERT for QnA**

In [91]:
!pip install transformers

In [93]:
import torch

In [94]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [95]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [96]:
def answer_question(question, answer_text):


    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

   
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

 
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [108]:
import textwrap


wrapper = textwrap.TextWrapper(width=80) 
file = open("/content/info.txt","r")
data = file.read()
def clean_text(data):
    """
    Removes non essential characters in corpus of text
    """
    data = "".join(v for v in data if v not in string.punctuation).lower()
    data = ''.join([i for i in data if not i.isdigit()])
    data = data.encode("utf8").decode("ascii",'ignore')
    return data

print(wrapper.fill(data[:1000]))

Sourdough bread is made by the fermentation of dough using naturally occurring
lactobacilli and yeast. It uses biological leavening rather than using
cultivated baker\'s yeast. The lactic acid produced by the lactobacilli gives it
a more sour taste and improved keeping qualities.[1]\nIn the Encyclopedia of
Food Microbiology, Michael Gaenzle writes: "The origins of bread-making are so
ancient that everything said about them must be pure speculation. One of the
oldest sourdough breads dates from 3700 BCE and was excavated in Switzerland,
but the origin of sourdough fermentation likely relates to the origin of
agriculture in the Fertile Crescent several thousand years earlier ... Bread
production relied on the use of sourdough as a leavening agent for most of human
history; the use of baker\'s yeast as a leavening agent dates back less than 150
years."[2]\nPliny the Elder described the sourdough method in his Natural
History:[3][4]\nGenerally however they do not heat it up at all, but onl

In [109]:
question = input("Ask a questions:")

answer_question(question, data[:1000])

Ask a questions:what does micheal gaenzie writes?
Query has 247 tokens.

Answer: "the encyclopedia of food microbiology"
